### A. Set up HMM model

In [3]:
states = ('Rainy', 'Sunny')
 
observations = ('walk', 'shop', 'clean')
 
initial_probability = {'Rainy': 0.6, 'Sunny': 0.4}
 
transition_matrix = {
   'Rainy' : {'Rainy': 0.7, 'Sunny': 0.3},
   'Sunny' : {'Rainy': 0.4, 'Sunny': 0.6},
   }
 
emission_matrix = {
   'Rainy' : {'walk': 0.1, 'shop': 0.4, 'clean': 0.5},
   'Sunny' : {'walk': 0.6, 'shop': 0.3, 'clean': 0.1},
   }

# modified from wiki: https://en.wikipedia.org/wiki/Hidden_Markov_model

In [12]:
emission_matrix['Rainy']['shop']

0.4

In [13]:
transition_matrix.keys()

['Rainy', 'Sunny']

In [15]:
states[:-1]

('Rainy',)

### B. Input a sequence of obervations vector

In [20]:
sequence_observations = ('shop', 'walk')

### C. Probablity of the sequence observation 

##### Forward Algorithm
Let $z_k$ be the $k$th hidden state that can take $m$ number of discrete values, e.g. ('Rainy', 'Sunny'), and $x_{1:k}=(x_1, x_2, ... , x_k)$ is the observation vector, where each $x_i$ is one of for example ('walk', 'shop', 'clean'). </br>

Compute $P(z_k, x_{1:k})$ first,</br>

$\begin{align*}
p(z_k, x_{1:k})&=\sum_{z_{k-1=1}}^{m}P(z_k. z_{k-1}, x_{1:k})\\
&=\sum_{z_{k-1}=1}P(x_k|z_k, z_{k-1}, x_{1:k-1})P(z_k|z_{k-1},x_{1:k-1})P(z_{k-1}|x_{1:k-1})P(x_{1:k-1})\\
\end{align*}$</br>

from Markov property,</br>

$\begin{equation*}
P(z_k, x_{1:k})=\sum_{z_{k-1}=1}P(x_k|z_k)P(z_k|z_{k-1})P(z_{k-1},x_{1:k-1})
\end{equation*}$</br>

notice how $P(z_k, x_{1:k})$ reoccurs, we can therefore apply recursion or dynamic programming, and we have the first term $P(z_1, x_{1})=P(z_1)P(x_1|z_1)$.

In [30]:
def forward_recur(z, x, tm=transition_matrix, em=emission_matrix, init=initial_probability):
    ''' z(str): state, e.g. rainny or sunny.
        x(tuple of strs): sequence of observation.
        tm(dict): transition matrix.
        em(dict): emission matrix.
        init(dict): initial probablity.'''
    m = tm.keys() # states
    if len(x) == 1:
        return em[z][x[0]]*init[z]
    pzx = 0.
    for i in range(len(m)):
        pzx += em[z][x[-1]]*tm[m[i]][z]*forward_recur(m[i], x[:-1])
    return pzx

# sum up joint dist
p_joint = [forward_recur(i, sequence_observations) for i in transition_matrix.keys()]
print "Probability of the observations vector is: ", sum(p_joint)

 Probability of the observations vector is:  0.108
